# Goal. 
create a single csv to programatically read i10ns strings from
## manual tweak required
1. "ARABIC FINAL" --> rename "ARABIC_FINAL"
2. Persian - delete some front-matter sheets in xlsx
3. replace all dates with '2019-08-02'	'2020-02-11' (with quotes, very stupid read_excel bug)

In [1]:
import pandas as pd
import os

In [2]:
pd.__version__

'1.0.4'

In [3]:
datadir = os.path.join(os.getenv('TRESORDIR'), 'CivilServant/projects/wikipedia-integration/gratitude-study/datasets/debrief-translations')

In [4]:
l10ns_f = os.listdir(datadir)

In [5]:
l10ns_f = [f for f in l10ns_f if f.endswith('.xlsx')]

In [6]:
l10ns_n_f = {f.split('_')[1]: f for f in l10ns_f}

In [7]:
l10ns_n_f

{'ARABIC': 'debrief matierials for translation_ARABIC_FINAL.xlsx',
 'GERMAN': 'debrief matierials for translation_GERMAN_FINAL.xlsx',
 'PERSIAN': 'debrief matierials for translation_PERSIAN_FINAL.xlsx',
 'POLISH': 'debrief matierials for translation_POLISH_FINAL.xlsx'}

In [8]:
l10ns_n_f

{'ARABIC': 'debrief matierials for translation_ARABIC_FINAL.xlsx',
 'GERMAN': 'debrief matierials for translation_GERMAN_FINAL.xlsx',
 'PERSIAN': 'debrief matierials for translation_PERSIAN_FINAL.xlsx',
 'POLISH': 'debrief matierials for translation_POLISH_FINAL.xlsx'}

In [9]:
thankee_l10ns = {n: pd.read_excel(os.path.join(datadir, f), sheet_name='Thankee') for n, f in l10ns_n_f.items()}
thanker_l10ns = {n: pd.read_excel(os.path.join(datadir, f), sheet_name='Thanker') for n, f in l10ns_n_f.items()}

In [10]:
thankee_l10ns['ARABIC']['{key} DO NOT TRANSLATE THIS COLUMN'].values

array(['table.1.title.left', 'table.1.title.right',
       'table.1.subtitle.left.1', 'table.1.subtitle.left.2',
       'table.1.subtitle.left.3', 'table.1.subtitle.left.4',
       'table.1.subtitle.right.1', 'table.1.subtitle.right.2',
       'table.1.subtitle.right.3', 'table.1.subtitle.right.4',
       'table.1.index.1', 'table.1.index.2', 'table.1.index.3',
       'table.1.index.4', 'table.1.index.5', 'table.1.index.6',
       'table.1.index.7', 'table.1.caption', nan, 'start.date',
       'end.date', 'end.caption.text', 'lab.anewc', 'lab.newc', 'lab.exp',
       nan, 'labor.hours.plot.title', 'labor.hours.plot.caption\n',
       'labor.hours.ylab', nan, 'retention.plot.ylab',
       'retention.plot.title ', 'retention.plot.caption', nan,
       'thanks.sent.ylab', 'thanks.sent.plot.title',
       'thanks.sent.plot.caption'], dtype=object)

In [11]:
print('start.date' in thankee_l10ns['ARABIC']['{key} DO NOT TRANSLATE THIS COLUMN'].values)

True


In [12]:
assert all([len(t.columns)==3 for t in thankee_l10ns.values()])

In [13]:
def clean_l10n_df_dict(l10ns_df_dict):
    clean_dfs = []
    for lang, df in l10ns_df_dict.items():
        df = df.iloc[:, :3] # select just first 2 columns
        print(lang)
        print('end.date' in df['{key} DO NOT TRANSLATE THIS COLUMN'].values)
        new_columns = [x if i!=2 else lang for i, x in enumerate(df.columns)] # make third column language name
        new_columns = [x.split('{')[1].split('}')[0] if '{' in x else x for x in new_columns] # remove curly braces if there
        new_columns = [x.lower() for x in new_columns] # lower cas
#         print(new_columns)
        df.columns = new_columns
        df = df[pd.notnull(df['key'])] # just the rows with keys
        df['key'] = df['key'].apply(lambda s: s.strip())
        clean_dfs.append(df)
    return clean_dfs

In [14]:
clean_thankee_l10ns = clean_l10n_df_dict(thankee_l10ns)
clean_thanker_l10ns = clean_l10n_df_dict(thanker_l10ns)

POLISH
True
ARABIC
True
GERMAN
True
PERSIAN
True
POLISH
False
ARABIC
False
GERMAN
False
PERSIAN
False


In [15]:
# next step combine in 5-wide df

In [16]:
langs = thankee_l10ns.keys()

In [17]:
clean_thankee_l10ns[1].head(20)

,key,english,arabic
0,table.1.title.left,eligible participants,المشاركون المؤهلون
1,table.1.title.right,average outcome values for accounts that were ...,متوسط قيم النتائج للحسابات التي لم يتم تخصيصها...
2,table.1.subtitle.left.1,monthly active,القائمة النشطة الشهرية
3,table.1.subtitle.left.2,available during the study,متوفرة خلال الدراسة
4,table.1.subtitle.left.3,included in the study,مدرجة في الدراسة
5,table.1.subtitle.left.4,% of treatment thanked,تم شكر % من العينة
6,table.1.subtitle.right.1,two week retention rate,معدل الاستبقاء لمدة أسبوعين
7,table.1.subtitle.right.2,pre-study labor hours,ساعات العمل قبل الدراسة
8,table.1.subtitle.right.3,difference in daily labor hours,الفرق في ساعات العمل اليومية
9,table.1.subtitle.right.4,thanks sent,تم إرسال الشكر


In [18]:
def make_wide(df_list):
    ret_df, rest_dfs = df_list[0], df_list[1:]
    for other_df in rest_dfs:
        ret_df = pd.merge(ret_df, other_df, on=['key', 'english'])
    return ret_df

In [19]:
thankee_wide = make_wide(clean_thankee_l10ns)

In [20]:
thankee_wide.shape

(33, 6)

In [21]:
thanker_wide = make_wide(clean_thanker_l10ns)

In [22]:
thanker_wide.shape

(24, 6)

In [23]:
# make the thanker one unique
thanker_wide['key'] = thanker_wide['key'].apply(lambda x: f'thanker.{x}')

In [24]:
# next step combine thanker and thankee into single df

In [25]:
fin_df = pd.concat([thankee_wide, thanker_wide])

In [26]:
fin_df.head(20)

,key,english,polish,arabic,german,persian
0,table.1.title.left,eligible participants,uczestnicy spełniający wymagania,المشاركون المؤهلون,geeignete Teilnehmer,شرکت کنندگان واجد شرایط
1,table.1.title.right,average outcome values for accounts that were ...,przypisani do otrzymania podziękowań (grupa ko...,متوسط قيم النتائج للحسابات التي لم يتم تخصيصها...,"durchschnittliche Ergebniswerte für Accounts, ...",میانگین متوسط نتيجه برای کاربرانی که تشکر برای...
2,table.1.subtitle.left.1,monthly active,miesięcznie aktywny,القائمة النشطة الشهرية,monatlich aktiv,فعال در طول ماه
3,table.1.subtitle.left.2,available during the study,dostępny podczas badania,متوفرة خلال الدراسة,während der Studie verfügbar,قابل دسترس در طول پژوهش
4,table.1.subtitle.left.3,included in the study,uwzględniony w badaniu,مدرجة في الدراسة,in die Studie aufgenommen,در پژوهش گنجانیده شده
5,table.1.subtitle.left.4,% of treatment thanked,% otrzymanych podziękowań,تم شكر % من العينة,% der Dankesempfänger in der Versuchsgruppe,درصد تجربه کسانی که مورد تشکر قرار گرفته شده اند
6,table.1.subtitle.right.1,two week retention rate,dwutygodniowy poziom utrzymania,معدل الاستبقاء لمدة أسبوعين,Bindungsrate über zwei Wochen,نرخ پایداری دو هفته
7,table.1.subtitle.right.2,pre-study labor hours,godziny robocze przed badaniem,ساعات العمل قبل الدراسة,Arbeitsstunden vor der Studie,ساعات کار پیش از پژوهش
8,table.1.subtitle.right.3,difference in daily labor hours,różnica w godzinach roboczych na dzień,الفرق في ساعات العمل اليومية,Differenz der Arbeitsstunden pro Tag,تفاوت در ميزان ساعات کار روزانه
9,table.1.subtitle.right.4,thanks sent,wysłane podziękowania,تم إرسال الشكر,verschickte Dankesnachrichten,ميزان تشکرهای ارسال شده


In [27]:
out_f = os.path.join(datadir, 'l10ns_thanker_thankee_all.csv')
fin_df.to_csv(out_f, index=False)

In [28]:
fin_df.shape

(57, 6)